# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

In [1]:
import openai
import os 

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client(
    api_key= os.getenv("OPENAI_API_KEY"),
    base_url= os.getenv("OPENAI_API_BASE_URL")
)

In [4]:
def get_completion(prompt, model="gpt-4o"):
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0.7
    )
    
    return response.choices[0].message.content

In [5]:
get_completion("What is 3.1415926535?")

"3.1415926535 is an approximation of the mathematical constant π (pi), which represents the ratio of a circle's circumference to its diameter. Pi is an irrational number, meaning it has an infinite number of decimal places without repeating. The value you provided is a common approximation used in various mathematical and scientific calculations."

In [6]:
customer_email = """
Arrr, I be funming that me blender lic\
flew off and splattered me kitchen walls\
with smoothie! And to make matters worse, \
the warranty don't cover the cost of\ 
clearning up me kitchen. I need yer help\
right now, matey!
""" 

In [7]:
style = """American English 
in a calm and respectful tone."""

In [8]:
prompt = f"""
Translate the text\ 
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
""" 

print(prompt)


Translate the text\ 
that is delimited by triple backticks 
into a style that is American English 
in a calm and respectful tone..
text: ```
Arrr, I be funming that me blender licflew off and splattered me kitchen wallswith smoothie! And to make matters worse, the warranty don't cover the cost of\ 
clearning up me kitchen. I need yer helpright now, matey!
```



In [9]:
get_completion(prompt)

"I'm quite frustrated that my blender malfunctioned and splattered smoothie all over my kitchen walls. To make matters worse, the warranty does not cover the cost of cleaning up the mess. I need your help right now."

here is a easy way

创建 ChatTemplate 实例 

给 ChatTemplate 实例 传入变量

用过 Chat 管道得到回复


In [10]:
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI # pip install -U langchain-openai

In [11]:
chat = ChatOpenAI(temperature=0.0, model="gpt-4o")
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10a21f510>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10a214610>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [12]:
template_string = """Translate the text\ 
that is delimited by triple backticks 
into a style that is {style}.
text: ```{text}```
"""

In [13]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [14]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text\\ \nthat is delimited by triple backticks \ninto a style that is {style}.\ntext: ```{text}```\n')

In [15]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [16]:
customer_messages = prompt_template.format_messages(style=style, text=customer_email)

In [17]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [18]:
print(customer_messages[0].content)

Translate the text\ 
that is delimited by triple backticks 
into a style that is American English 
in a calm and respectful tone..
text: ```
Arrr, I be funming that me blender licflew off and splattered me kitchen wallswith smoothie! And to make matters worse, the warranty don't cover the cost of\ 
clearning up me kitchen. I need yer helpright now, matey!
```



In [19]:
customer_response = chat(customer_messages)
print(customer_response.content)

/Users/guchen/repo/LLM-complete/.conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Sure, here is the text translated into American English in a calm and respectful tone:

```
I am quite upset that my blender malfunctioned and splattered smoothie all over my kitchen walls. To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I need your help right now, please.
```


In [21]:
# let's see another example
service_reply = """ 
Hey there customer, \
the warranty dose not cover \
cleaning expense for yoru kitchen\
because it's your fault that\
you misused your blender\
by forgetting to put the lid on before\
starting the blender.\
Tough luck! See ya!
"""

In [22]:
service_style_pirate = """\
    a polite tone\
        that speaks a English Pirate\
    """

In [23]:
service_message = prompt_template.format_messages(
    style=service_style_pirate, 
    text=service_reply
)

print(service_message[0].content)

Translate the text\ 
that is delimited by triple backticks 
into a style that is     a polite tone        that speaks a English Pirate    .
text: ``` 
Hey there customer, the warranty dose not cover cleaning expense for yoru kitchenbecause it's your fault thatyou misused your blenderby forgetting to put the lid on beforestarting the blender.Tough luck! See ya!
```



In [24]:
service_response = chat(service_message)
print(service_response.content)

Aye, me hearty customer,

I regret to inform ye that the warranty does not cover the cost of cleanin' yer galley, as it be due to yer own misadventure in usin' the blender. It seems ye forgot to secure the lid before settin' sail with the blender. Alas, such be the way of things. Fair winds to ye!

Yours in service,
[Your Name]


In [25]:
# extract customer review into a json value

customer_review = """\
This leaf blower is pretty amazing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado.\
It arrived in two days, just in time for my wife's\
anniversary present.\
I think my wife liked it so much she was speechless.\
So far I've been the only one using it. and I've been using it every other morning to clear the leaves on our lawern.\
It's slightly more expensive than the other leaf blowers\
out there, but I think it's worth it for the extra features. 
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as gift for someone else?  Answer True if yes, False if not or unknown.
delivery_datys : How many days did it take for the item to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price. and output them as comma separated Python list.

Format the output as JSON with the following keys:
gift : bool
delivery_days : number
price_value : List[str]

text: {text}
"""

In [26]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as gift for someone else?  Answer True if yes, False if not or unknown.\ndelivery_datys : How many days did it take for the item to arrive? If this information is not found, output -1.\nprice_value: Extract any sentences about the value or price. and output them as comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift : bool\ndelivery_days : number\nprice_value : List[str]\n\ntext: {text}\n'))]


In [27]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages)
print(response.content)
print(type(response.content))

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```
<class 'str'>


In [28]:
# using langchain parsers

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [29]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [30]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [31]:
format_instuctions = output_parser.get_format_instructions()
print(format_instuctions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [32]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instuctions)

In [33]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing. It has four settings:candle blower, gentle breeze, windy city, and tornado.It arrived in two days, just in time for my wife'sanniversary present.I think my wife liked it so much she was speechless.So far I've been the only one using it. and I've been using it every other morning to clear the leaves on our lawern.It's slightly more expensive than the other leaf blowersout there, but I think it's worth it for the extra features. 


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":



In [34]:
response = chat(messages)
print(response.content)
print(type(response.content))

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```
<class 'str'>


In [35]:
output_dict = output_parser.parse(response.content)
print(output_dict)
print(type(output_dict))

{'gift': 'True', 'delivery_days': '2', 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}
<class 'dict'>


In [36]:
output_dict.get("delivery_days")

'2'